In [65]:
import pandas as pd
import numpy as np

In [66]:
cafe = pd.read_csv('dirty_cafe_sales.csv')
cafe.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


In [67]:
cafe.info()
cafe.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64

In [68]:
#Make column names more convenient
#Приводим название колонок к удобному виду
cafe.columns = [c.strip().lower().replace(" ", "_") for c in cafe.columns]


In [69]:
#We replace garbage values ​​with gaps
#Заменяем мусорные значения на пропуски
bad = {"ERROR": np.nan, "UNKNOWN": np.nan, "": np.nan, " ": np.nan}
cafe = cafe.replace(bad)


In [70]:
cafe.isna().sum()

transaction_id         0
item                 969
quantity             479
price_per_unit       533
total_spent          502
payment_method      3178
location            3961
transaction_date     460
dtype: int64

In [71]:
#Converting "numeric" columns into numbers
#Превращаем “числовые” колонки в числа
cafe["quantity"] = pd.to_numeric(cafe["quantity"], errors="coerce")
cafe["price_per_unit"] = pd.to_numeric(cafe["price_per_unit"], errors="coerce")
cafe["total_spent"] = pd.to_numeric(cafe["total_spent"], errors="coerce")

In [72]:
#Converting a date to a normal date format
#Превращаем дату в нормальный формат даты
cafe["transaction_date"] = pd.to_datetime(cafe["transaction_date"], errors="coerce")



In [73]:
#Removing duplicate transactions
#Убираем дубликаты транзакций
cafe = cafe.drop_duplicates(subset=["transaction_id"])

In [74]:
#Recalculating total_spent
#Пересчитываем total_spent
calc_total = cafe["quantity"] * cafe["price_per_unit"]
need_fix = cafe["total_spent"].isna() | (cafe["total_spent"] <= 0)
cafe.loc[need_fix, "total_spent"] = calc_total


In [75]:
#Zero/negative values ​​are considered an error
#Нулевые/отрицательные значения считаем ошибкой
cafe.loc[cafe["quantity"] <= 0, "quantity"] = np.nan
cafe.loc[cafe["price_per_unit"] <= 0, "price_per_unit"] = np.nan
cafe.loc[cafe["total_spent"] <= 0, "total_spent"] = np.nan


In [76]:
#Clearing spaces from text fields
#Чистим текстовые поля от пробелов
cafe["item"] = cafe["item"].astype(str).str.strip().replace({"nan": np.nan})
cafe["payment_method"] = cafe["payment_method"].astype(str).str.strip().replace({"nan": np.nan})
cafe["location"] = cafe["location"].astype(str).str.strip().replace({"nan": np.nan})


In [77]:
#We delete lines where key information was not restored.
#Удаляем строки, где ключевая информация так и не восстановилась
cafe = cafe.dropna(subset=["transaction_id", "transaction_date", "item", "quantity", "price_per_unit", "total_spent"])

In [78]:
#Rounding up money
#Округляем деньги
cafe["total_spent"] = cafe["total_spent"].round(2)
cafe["price_per_unit"] = cafe["price_per_unit"].round(2)

In [79]:
cafe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7773 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    7773 non-null   object        
 1   item              7773 non-null   object        
 2   quantity          7773 non-null   float64       
 3   price_per_unit    7773 non-null   float64       
 4   total_spent       7773 non-null   float64       
 5   payment_method    5319 non-null   object        
 6   location          4679 non-null   object        
 7   transaction_date  7773 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 546.5+ KB


In [82]:
cafe.to_csv("clean_cafe_sales", index=False)
cafe.head()


,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2.0,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4.0,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4.0,1.0,4.0,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2.0,5.0,10.0,NaN,NaN,2023-04-27
4,TXN_3160411,Coffee,2.0,2.0,4.0,Digital Wallet,In-store,2023-06-11
